# Optimising my yearly savings planning with PuLP

## Importing packages

In [14]:
import pulp
import pandas as pd
import numpy as np

## Importing data

In [41]:
df_savings = pd.read_excel('2020.xlsx', sheet_name = 'savings')
df_savings

,category,target
0,student_loan,1300
1,drivers_license,700
2,emergencies,2000
3,israel,1000
4,munich,300
5,peru,1500
6,estate,7000


In [42]:
df_expenses = pd.read_excel('2020.xlsx', sheet_name = 'expenses')
df_expenses

,expenses,value
0,rent,695
1,joint_account,500
2,student_loan,126
3,services,100


Let's define the time period that covers this problem:

In [169]:
duration = 12
months = range(1,duration)
print(months)

range(1, 12)


$$E_m$$ is a vector containing the monthly expenses, which are the sum of the categories. 

In [127]:
expenses = list(np.repeat(sum(df_expenses['value']), duration))

print('Monthly expenses E : ' + str(expenses))

Monthly expenses E : [1421, 1421, 1421, 1421, 1421, 1421, 1421, 1421, 1421, 1421, 1421, 1421]


In [128]:
monthly_salary = 2100
revenue = list(np.repeat(monthly_salary, duration))

revenue[5]=revenue[5]+800 #participation en mai
revenue[11]=revenue[11]*2 #treizieme mois

print('Monthly revenue R : ' + str(revenue))

Monthly revenue R : [2100, 2100, 2100, 2100, 2100, 2900, 2100, 2100, 2100, 2100, 2100, 4200]


In [53]:
s_target = df_savings

In [49]:
mysavings = pulp.LpProblem("My savings planning", pulp.LpMaximize)

## Defining the decision variables

Our goal is to maximise the balance, after deducing the expenses and savings from the initial revenue.

$$ m $$ represents the number of months over which the savings are planned

$$max(B^m) = max(R^m - E^m - D^m - \sum_jS^m)$$

Each of these decision variables will have similar characteristics (lower bound of 0, integer variables). Therefore we can use PuLP’s LpVariable object’s dict functionality in order to characterize them. Other categories of variables include Integer and Binary.
Savings are mysavingsled as an integer variable (in order to restrain the problem).

In [76]:
# How much money should be allocated each month as savings for each category of savings

savings = pulp.LpVariable.dicts("Savings",
                                     ((m,j) for m in months for j in s_target.category),
                                     lowBound=0,
                                     cat='Integer')

In [77]:
# How much money should be used each month

daily = pulp.LpVariable.dicts("Daily expenses",
                                     ((m) for m in months),
                                     lowBound=0,
                                     cat='Integer')

## Building the objective function

PuLP provides an lpSum vector calculation for the sum of a list of linear expressions.
List comprehensions (in brackets) allow us to scale up the problem to several variables, instead of defining each element of the vector.

In [200]:
r = [revenue[m] for m in months]      #monthly revenue

e = [expenses[m] for m in months] #monthly expenses
d = [daily[m] for m in months] #daily expenses

s = [savings[m,j] for m in months for j in s_target['category']] #monthly savings per category

In [201]:
mysavings = pulp.lpSum(r)- pulp.lpSum(e) - pulp.lpSum(d) - pulp.lpSum(s)

## Establishing the problem constraints

The main constraint is the revenue. Revenue is (unfortunately) fixed per month in a year but there are bonuses in June and December : 

In [ ]:
# Constraints
# 350 economy and 500 premium sausages at 0.05 kg
mysavings += pulp.lpSum([ing_weight['economy', j] for j in ingredients]) == 350 * 0.05
mysavings += pulp.lpSum([ing_weight['premium', j] for j in ingredients]) == 500 * 0.05

# Economy has >= 40% pork, premium >= 60% pork
mysavings += ing_weight['economy', 'pork'] >= (
    0.4 * pulp.lpSum([ing_weight['economy', j] for j in ingredients]))

mysavings += ing_weight['premium', 'pork'] >= (
    0.6 * pulp.lpSum([ing_weight['premium', j] for j in ingredients]))

# Sausages must be <= 25% starch
mysavings += ing_weight['economy', 'starch'] <= (
    0.25 * pulp.lpSum([ing_weight['economy', j] for j in ingredients]))

mysavings += ing_weight['premium', 'starch'] <= (
    0.25 * pulp.lpSum([ing_weight['premium', j] for j in ingredients]))

# We have at most 30 kg of pork, 20 kg of wheat and 17 kg of starch available
mysavings += pulp.lpSum([ing_weight[i, 'pork'] for i in sausage_types]) <= 30
mysavings += pulp.lpSum([ing_weight[i, 'wheat'] for i in sausage_types]) <= 20
mysavings += pulp.lpSum([ing_weight[i, 'starch'] for i in sausage_types]) <= 17

# We have at least 23 kg of pork to use up
mysavings += pulp.lpSum([ing_weight[i, 'pork'] for i in sausage_types]) >= 23

## Solving the problem !

In [195]:
# Solve our problem
mysavings.solve()
pulp.LpStatus[mysavingsplanning.status]

AttributeError: 'LpAffineExpression' object has no attribute 'status'

We have also checked the status of the solver, there are 5 status codes:
- Not Solved: Status prior to solving the problem.
- Optimal: An optimal solution has been found.
- Infeasible: There are no feasible solutions (e.g. if you set the constraints x <= 1 and x >=2).
- Unbounded: The constraints are not bounded, maximising the solution will tend towards infinity (e.g. if the only constraint was x >= 3).
- Undefined: The optimal solution may exist but may not have been found.

We can use the varValue method to retrieve the values of our variables x and y, and the pulp.value function to view the maximum value of the objective function.

In [ ]:
for var in ing_weight:
    var_value = ing_weight[var].varValue
    print "The weight of {0} in {1} sausages is {2} kg".format(var[1], var[0], var_value)

In [ ]:
total_cost = pulp.value(mysavings.objective)

print "The total cost is €{} for 350 economy sausages and 500 premium sausages".format(round(total_cost, 2))